In [24]:
from dotenv import dotenv_values
import praw

In [ ]:
config = dotenv_values(".env")

In [32]:
reddit = praw.Reddit(client_id=config['client_id'], client_secret=config['client_secret'], user_agent=config['user_agent'])

In [ ]:
import pandas as pd
subreddit = reddit.subreddit('travel')
search_query = 'United States'

submissions = subreddit.search(query=search_query, sort='top', time_filter='all', limit=200)
thread_ids = []
thread_titles = []
thread_urls = []
thread_contents = []
thread_name = []
thread_author = []
thread_created_utc = []
thread_subreddit_id = []
thread_subreddit = []
comments_list = []

for submission in submissions:
    thread_ids.append(submission.id)
    thread_subreddit.append(submission.subreddit)
    thread_name.append(submission.name)
    thread_author.append(submission.author.name if submission.author else '[deleted]',)
    thread_titles.append(submission.title)
    thread_contents.append(submission.selftext)
    thread_urls.append(submission.url)
    thread_created_utc.append(submission.created_utc)
    thread_subreddit_id.append(submission.subreddit_id)
    
    submission.comments.replace_more(limit=10)
    comments = submission.comments.list()

    for comment in comments:
        comments_list.append({
            'id': comment.id,
            'post_id': submission.id,
            'post_title': submission.title,
            'comment': comment.body,
            'author': comment.author.name if comment.author else '[deleted]',
            'created_at': comment.created_utc
        })


post_data = {
    'id': thread_ids,
    'title': thread_titles,
    'url': thread_urls,
    'content': thread_contents,
    'name': thread_name,
    'author': thread_author,
    'created_at': thread_created_utc,
    'subreddit': thread_subreddit,
    'subreddit_id': thread_subreddit_id
}

post_df = pd.DataFrame(post_data)

comment_df = pd.DataFrame(comments_list)

post_df.to_csv('travel_posts_data.csv', index=False)
comment_df.to_csv('travel_comments_data.csv', index=False)

print("Data has been successfully saved to 'travel_posts_data.csv' and 'travel_comments_data.csv'.")
